In [2]:
# Import packages. If unable to import, try "!pip install package" first.
# Selenium package is required. This package has to be used with Google Chrome Driver.
# Here is a tutorial: https://blog.csdn.net/qq_48736958/article/details/115179198

import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import os
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.support.ui as ui
import pandas as pd
import sys,time
from tqdm import tqdm
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import json
import ast
import re

In [3]:
# Define several functions to be used later
def clean_Table_of_Contents(lst):
    clean_lst = []
    for item in lst:
        # Remove weird symbols like \u2002
        clean_item = re.sub(r'[\u2000-\u200a]', '', item)
        clean_item = clean_item.strip()
        clean_item = re.sub(r'(\s|^)\d+$', '', clean_item)
        clean_lst.append(clean_item.strip())
    
    clean_lst = [item for item in clean_lst if item != ""]
    return clean_lst

def find_element_index(lst, target_word):
    for i, item in enumerate(lst):
        if target_word in item.lower():
            return i
    return None

In [4]:
URL_list = [
"https://www.sec.gov/ix?doc=/Archives/edgar/data/701288/000110465924044922/tm242747d4_def14a.htm",
"https://www.sec.gov/Archives/edgar/data/775368/000120677421000710/y3856321-def14a.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/1475922/000119312524078288/d775324ddef14a.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/6201/000119312524114008/d636721ddef14a.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/3197/000155837024004904/ceco-20240520xdef14a.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/764622/000076462224000033/pnw-20240404.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/2178/000000217824000043/ae-20240327.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/766421/000095017024038178/alk-20240328.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/701288/000110465924044922/tm242747d4_def14a.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/3453/000155837024002901/matx-20240425xdef14a.htm",
"https://www.sec.gov/Archives/edgar/data/775368/000120677421000710/y3856321-def14a.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/773840/000077384024000032/hon-20240402.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/4447/000119312524088446/d520445ddef14a.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/1475922/000119312524078288/d775324ddef14a.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/4904/000000490424000023/aep-20240313.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/4962/000119312524068837/d558747ddef14a.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/5272/000000527224000034/aig-20240402.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/5981/000095017024048460/avd-20240425.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/1037868/000103786824000022/ame-20240312.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/318154/000119312524099062/d445034ddef14a.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/821026/000082102624000108/ande-20240313.htm",
"https://www.sec.gov/Archives/edgar/data/7039/000000703921000041/trecora-2021proxystatement.htm",
"https://www.sec.gov/ix?doc=/Archives/edgar/data/7084/000119312524091968/d540412ddef14a.htm"]

In [5]:
# Activate Chrome Driver.
options = Options()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-blink-features")
##options.add_argument("--proxy-server=https://pr.oxylabs.io:7777")
driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})
driver.maximize_window()

In [6]:
Outputs = []
for Proxy_statement_url in tqdm(URL_list):
    ## First of all, we need to reconstrurct the url so that the web page could be scraped
    Proxy_statement_url_revised = Proxy_statement_url.replace("ix?doc=/","")
    driver.get(Proxy_statement_url_revised)




    ## Secondly, we need to locate the Table of Content and obtain all headlines.

    # Obtain all possible tables in the file
    # Locate the one with keywords like "pay ratio"
    # The table with our keywords is most likely to be "Table of Content"
    list_of_tables = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.XPATH, ".//table")))
    for table in list_of_tables:
        table_of_content = table.text.lower()
        if "pay ratio" in table_of_content:            
            break
    
    if "pay ratio" not in table_of_content:
        Outputs.append({"URL":Proxy_statement_url,"Pay_Ratio_Paragraph":"Need Manual Work","Reasons":"Table of Content Not Found"})
        print("Table of Content Not Found")
        continue
            
    TOC_list = clean_Table_of_Contents(table_of_content.split("\n"))

    # Obtain our target headline and the headline next to our target headline within "Table of Content"
    Target_headline_index = find_element_index(TOC_list, "pay ratio")
    Next_headline_index = Target_headline_index + 1
    Target_headline = TOC_list[Target_headline_index]
    Next_headline = TOC_list[Next_headline_index]




    ## Thirdly, Obtain the full text of the file. And Extract the Paragraph of "Pay Ratio"

    # Obtain the full text of the file
    Full_text = driver.find_element(By.XPATH, "//body").text.lower()

    # Obtain the text of "Table of Content"
    TOC_text = table_of_content

    # Remove the text of "Table of Content" from the full text. Convert all paragraphs into list.
    Text_of_File = Full_text.replace(TOC_text,"").split("\n")

    # Remove space at the front and at the end of each paragraph. (This can facilitate our matching later)
    Text_of_File = [item.strip().lower() for item in Text_of_File]

    # Locate the position of the two headlines within the paragraphs and extract the text
    try:
        Target_headline_index_2 = Text_of_File.index(Target_headline.lower())
        Next_headline_index_2 = Text_of_File[Target_headline_index_2:].index(Next_headline.lower())
        Pay_Ratio_Paragraph = "\n".join(Text_of_File[Target_headline_index_2:Target_headline_index_2 + Next_headline_index_2])
    except:
        Outputs.append({"URL":Proxy_statement_url,"Pay_Ratio_Paragraph":"Need Manual Work","Reasons":"Headline Not Found"})
        print("Headline Not Found")
        continue
    
    
    ## Finally, Organize the results and append them in a list
    Outputs.append({"URL":Proxy_statement_url,"Pay_Ratio_Paragraph":Pay_Ratio_Paragraph})
    
    


 22%|██▏       | 5/23 [00:21<01:19,  4.41s/it]

Headline Not Found


 30%|███       | 7/23 [00:30<01:07,  4.23s/it]

Table of Content Not Found


 43%|████▎     | 10/23 [00:43<00:58,  4.52s/it]

...

 91%|█████████▏| 21/23 [01:27<00:07,  3.83s/it]

Headline Not Found


 96%|█████████▌| 22/23 [01:29<00:03,  3.39s/it]

Headline Not Found


100%|██████████| 23/23 [01:36<00:00,  4.18s/it]


In [8]:
pd.DataFrame(Outputs).to_excel(r"C:\Users\admin\Downloads\Testing Outputs.xlsx", index=False)

In [10]:
print(pd.DataFrame(Outputs).head())

                                                 URL  \
0  https://www.sec.gov/ix?doc=/Archives/edgar/dat...   
1  https://www.sec.gov/Archives/edgar/data/775368...   
2  https://www.sec.gov/ix?doc=/Archives/edgar/dat...   
3  https://www.sec.gov/ix?doc=/Archives/edgar/dat...   
4  https://www.sec.gov/ix?doc=/Archives/edgar/dat...   

                                 Pay_Ratio_Paragraph             Reasons  
0  pay ratio disclosure\npursuant to item 402(u) ...                 NaN  
1  ceo pay ratio\ngeneral\nwe are providing the f...                 NaN  
2  pay ratio\nin august 2015, pursuant to a manda...                 NaN  
3  ceo pay ratio\nas required by section 953(b) o...                 NaN  
4                                   Need Manual Work  Headline Not Found  
